<a href="https://colab.research.google.com/github/Geolist/first-repository/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코사인 유사도

## NumPy 활용

In [1]:
import numpy as np

t1 = np.array([1, 1, 1])
t2 = np.array([2, 0, 1])

In [2]:
print(t1)

[1 1 1]


In [3]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
	return dot(A, B)/(norm(A)*norm(B))

In [4]:
cos_sim(t1, t2)

np.float64(0.7745966692414834)

## 사이킷런 활용

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
t1 = np.array([[1, 1, 1]])
t2 = np.array([[2, 0, 1]])
cosine_similarity(t1,t2)

array([[0.77459667]])

In [7]:
# Q. 아래 t1, t2 리스트의 요소 값을 조정해서 코사인 유사도 값이 0이 되게 만들어보세요.
# +코사인 유사도 값이 -1이 되게도 만들어보세요.
t1 = np.array([[1, 1, 1]])
t2 = np.array([[-1, 0, 1]])
cosine_similarity(t1,t2)

array([[0.]])

In [8]:
# Q. 아래 t1, t2 리스트의 요소 값을 조정해서 코사인 유사도 값이 0이 되게 만들어보세요.
# +코사인 유사도 값이 -1이 되게도 만들어보세요.
t1 = np.array([[1, 1, 1]])
t2 = np.array([[-1, -1, -1]])
cosine_similarity(t1,t2)

array([[-1.]])

# 콘텐츠 기반 필터링 실습

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print('⏩⏩')

⏩⏩


In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 데이터 로드 movie_dataset

In [11]:
# 파일 경로 지정
csv_path = '/content/drive/My Drive/movie_dataset.csv'

# CSV 읽기
df = pd.read_csv(csv_path)

# 데이터 미리보기
df.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [12]:
# 데이터프레임 shape
print(df.shape)

# 컬럼명 확인
print(df.columns)

(4803, 24)
Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')


In [13]:
features = ['keywords','cast','genres','director']
features

['keywords', 'cast', 'genres', 'director']

In [14]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

combine_features(df[:5])

,0
0,culture clash future space war space colony so...
1,ocean drug abuse exotic island east india trad...
2,spy based on novel secret agent sequel mi6 Dan...
3,dc comics crime fighter terrorist secret ident...
4,based on novel mars medallion space travel pri...


In [15]:
combined = combine_features(df.iloc[0])
print(combined)


culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Action Adventure Fantasy Science Fiction James Cameron


In [16]:
for feature in features:
    df[feature] = df[feature].fillna('')

df["combined_features"] = df.apply(combine_features,axis=1)
df["combined_features"]

,combined_features
0,culture clash future space war space colony so...
1,ocean drug abuse exotic island east india trad...
2,spy based on novel secret agent sequel mi6 Dan...
3,dc comics crime fighter terrorist secret ident...
4,based on novel mars medallion space travel pri...
...,...
4798,united states\u2013mexico barrier legs arms pa...
4799,Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800,date love at first sight narration investigati...
4801,Daniel Henney Eliza Coupe Bill Paxton Alan Ru...


##  벡터화하고, 코사인 유사도를 계산

In [17]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print(type(count_matrix))
print(count_matrix.shape)
print(count_matrix)

<class 'scipy.sparse._csr.csr_matrix'>
(4803, 14845)
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 97547 stored elements and shape (4803, 14845)>
  Coords	Values
  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


In [18]:
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)
print(cosine_sim.shape)


[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]
(4803, 4803)


In [19]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 3편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==3:
        break

Avatar와 비슷한 영화 3편은 

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1


In [20]:
# 1. index 컬럼이 없는 경우 대비
if "index" not in df.columns:
    df["index"] = df.index

# 제목 → index
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

# index → 제목
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

# 찾고 싶은 영화
movie_user_likes = "Titanic"

# Titanic의 index
movie_index = get_index_from_title(movie_user_likes)

# Titanic과 모든 영화 간 유사도 벡터
similar_movies = list(enumerate(cosine_sim[movie_index]))

# 유사도 기준 내림차순 정렬
sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:]

# 상위 5편 출력
print(movie_user_likes + "과(와) 유사한 영화 5편은:\n")

i = 0
for item in sorted_similar_movies:
    title = get_title_from_index(item[0])
    similarity = item[1]
    print(f"{title} (유사도: {similarity:.3f})")
    i += 1
    if i == 5:
        break


Titanic과(와) 유사한 영화 5편은:

Revolutionary Road (유사도: 0.300)
Me You and Five Bucks (유사도: 0.258)
All the King's Men (유사도: 0.244)
The Day the Earth Stood Still (유사도: 0.238)
Almost Famous (유사도: 0.238)


## 협업 필터링 (1) 협업 필터링의 종류

## 협업 필터링 (2) 행렬 인수분해

In [21]:
import numpy as np
from numpy.linalg import svd

In [22]:
np.random.seed(30)
A = np.random.randint(0, 100, size=(4, 4))
A

array([[37, 37, 45, 45],
       [12, 23,  2, 53],
       [17, 46,  3, 41],
       [ 7, 65, 49, 45]])

In [23]:
svd(A)

SVDResult(U=array([[-0.54937068, -0.2803037 , -0.76767503, -0.1740596 ],
       [-0.3581157 ,  0.69569442, -0.13554741,  0.60777407],
       [-0.41727183,  0.47142296,  0.28991733, -0.72082768],
       [-0.6291496 , -0.46389601,  0.55520257,  0.28411509]]), S=array([142.88131188,  39.87683209,  28.97701433,  14.97002405]), Vh=array([[-0.25280963, -0.62046326, -0.4025583 , -0.6237463 ],
       [ 0.06881225, -0.07117038, -0.8159854 ,  0.56953268],
       [-0.73215039,  0.61782756, -0.23266002, -0.16767299],
       [-0.62873522, -0.47775436,  0.34348792,  0.50838848]]))

In [24]:
U, Sigma, VT = svd(A)

print('U matrix: {}\n'.format(U.shape),U)
print('Sigma: {}\n'.format(Sigma.shape),Sigma)
print('V Transpose matrix: {}\n'.format(VT.shape),VT)

U matrix: (4, 4)
 [[-0.54937068 -0.2803037  -0.76767503 -0.1740596 ]
 [-0.3581157   0.69569442 -0.13554741  0.60777407]
 [-0.41727183  0.47142296  0.28991733 -0.72082768]
 [-0.6291496  -0.46389601  0.55520257  0.28411509]]
Sigma: (4,)
 [142.88131188  39.87683209  28.97701433  14.97002405]
V Transpose matrix: (4, 4)
 [[-0.25280963 -0.62046326 -0.4025583  -0.6237463 ]
 [ 0.06881225 -0.07117038 -0.8159854   0.56953268]
 [-0.73215039  0.61782756 -0.23266002 -0.16767299]
 [-0.62873522 -0.47775436  0.34348792  0.50838848]]


In [25]:
Sigma_mat = np.diag(Sigma)

A_ = np.dot(np.dot(U, Sigma_mat), VT)
A_

array([[37., 37., 45., 45.],
       [12., 23.,  2., 53.],
       [17., 46.,  3., 41.],
       [ 7., 65., 49., 45.]])